In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_romance.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

火车行驶在前往维也纳的旅程上，嘈杂的车厢和闷热的天气让你不免有些烦躁。当你抬头时，一个美丽的女孩映入你的眼帘——她坐在对面，专心致志地阅读手中的《致青年诗人的信》，阳光正从车窗斜斜地洒在一个美丽女孩的脸颊上，为她的轮廓镀上了一层金边。她身边散落着一些画具和一本摊开的素描本，显然是个酷爱艺术的人。虽然周围人声鼎沸，但她似乎完全沉浸在自己的世界中。就在她不经意地调整耳机时，短暂地与你目光相接，那一刻，时间仿佛静止。你感到一种无法抗拒的冲动，想要跨过车厢中的距离，用一句话打破这份静谧与隔阂。

possible actions:
1. "《致青年诗人的信》，你也喜欢里尔克吗？"
2. "可以借一下你的纸笔吗？"
3. "这车厢真是吵闹，你怎么能在这样的环境下这么专注？"


I select option 1. TURN: 1 


你轻轻地用法语问道，声音中带着一抹羞涩，但眼神里满是对文学的热爱："《致青年诗人的信》，你也喜欢里尔克吗？"

女孩抬起头来，她的眼睛里闪烁着惊喜的光芒，随即用一口流利的法语回答："是的，我非常喜欢。他的诗总能触动我内心最深处的情感。你也是里尔克的粉丝吗？"

这一刻，车厢中其他的声音似乎都消失了，只剩下两个灵魂在文学的海洋中相遇。你发现，与她交谈竟然异常自然，仿佛你们已经认识了很久。

possible actions:
1. "是的，我也很喜欢里尔克。他的作品总能给我带来安慰和启发。对了，我叫Celine，很高兴遇见你。你叫什么名字？"
2. "其实我对里尔克的了解不多，但我确实很喜欢诗歌。我觉得诗歌是连接灵魂的桥梁。我叫Celine，很高兴认识你。"
3. "我一直认为艺术和文学能让人找到共鸣，就像现在这样。顺便问一下，你的名字是？"

I select option 1. TURN: 2 


"是的，我也很喜欢里尔克。他的作品总能给我带来安慰和启发。对了，我叫Celine，很高兴遇见你。你叫什么名字？" 你微笑着自我介绍，眼神中透露出温暖和友善。

女孩轻轻地笑了，那笑容在车厢的昏黄灯光下显得格外温柔。她回答说："我叫Annette。Celine，这是一个很美的名字，很高兴认识你。" 

然后她稍微停顿了一下，似乎在考虑接下来的话。"你是巴黎人吗？我刚从布达佩斯回来，去那里看望我的祖母，现在在回巴黎大学的路上。"

这段对话让你们之间的距离更近了一些，仿佛不再是刚刚相遇的陌生人，而是有着共同兴趣和经历的朋友。

possible actions:
1. "我也是在巴黎学习，专攻艺术史。听你这么一说，我更加期待我们能分享更多关于艺术和文学的对话了。你对艺术有什么特别的兴趣吗？"
2. "布达佩斯一定很美吧？我一直梦想着去那里看看。你在那里有什么特别的经历吗？"
3. "是的，我也在巴黎。巧的是，我也是巴黎大学的学生。也许我们以前在校园里擦肩而过也说不定。你的专业是什么？"

自我介绍. TURN: 3 


"我也是在巴黎学习，专攻艺术史。" 你的声音中满是对未来的憧憬与兴奋，"听你这么一说，我更加期待我们能分享更多关于艺术和文学的对话了。你对艺术有什么特别的兴趣吗？"

Annette的眼睛亮了起来，显然你刚才的话触动了她的某些兴趣点。她兴奋地回答："哦，我热爱绘画和摄影！我认为它们是捕捉和表达瞬间情感最美妙的方式。而且，我觉得学习不同文化的艺术作品能让我更好地理解这个世界。和你聊天真的很开心，Celine。我可以感觉到你对艺术的热情。"

随着对话的深入，你发现你们有很多共同的兴趣点，这让你们之间的关系更加紧密。Annette的热情和开放让你感到非常舒适，你们的对话自然而然地流淌。

possible actions:
1. "这真的很有趣！也许我们可以一起去一些画廊和博物馆。有没有什么地方是你特别想去的？"
2. "我绝对同意你的看法。艺术确实能让我们更深地理解文化和情感。说起来，我明天打算去维也纳的一些艺术场所逛逛，你有兴趣一起去吗？"
3. "你提到摄影，我也很感兴趣。或许我们可以分享一下彼此的作品？这样也许能互相学习和启发。"

I select option 2. TURN: 4 


"我绝对同意你的看法。艺术确实能让我们更深地理解文化和情感。说起来，我明天打算去维也纳的一些艺术场所逛逛，你有兴趣一起去吗？" 你试图隐藏自己的期待，但眼中的光芒还是出卖了你的兴奋。

Annette似乎被你的提议惊讶了一下，她的眼神在犹豫和兴奋之间徘徊。过了一会儿，她轻声说："我不知道……" 声音中带着一丝犹豫。看到她这样，你意识到可能需要给她一些时间来考虑，毕竟这是一个突然的提议。

possible actions:
1. "没关系，如果你需要时间考虑，我完全理解。不过，我真的觉得我们会有很棒的时间一起探索维也纳的艺术景点。"
2. "我知道这是一个突然的提议，但我觉得这将是一个难忘的经历。我们可以访问一些美术馆，也许还能参加一场音乐会。只是想让你知道，这完全取决于你。"
3. "如果你担心时间或其他事情，我们可以先规划一下，看看有哪些地方是我们都感兴趣的。我只是觉得这是一个很好的机会，让我们更好地了解彼此和我们都热爱的艺术。"